In [2]:
import subprocess
from pyspark.sql.types import IntegerType,FloatType
from functools import reduce
from pyspark.sql import DataFrame,SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import sum

#Intializing SparkSession and configuration for spark.config can also set here 
conf = SparkConf().setMaster("local").setAppName("test")
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
# Function that is created to to read the hdfs file using SparkSession
def hdfs_file_read(file):
    rdd = spark.read.csv(file,header=True,inferSchema = True)
    return rdd

In [4]:
#Function performs dataset computation and return dataframe for normalWeek and lockdownWeek  
def get_Dataframe(x):
    p=subprocess.Popen(x,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT)
    result=list()
    for line in p.stdout.readlines():
        file = line.decode().strip()
        df= hdfs_file_read(file)
        check_columns=df.select('class','classname').distinct().sort('class',ascending=True)
        #remove the null rows
        speed_df=df.na.drop(subset=['classname'])
        speed_df=speed_df.groupBy('class','classname').avg('speed').sort('class',ascending=True)
        speed_df=speed_df.withColumnRenamed("avg(speed)", "speed")
        result.append(speed_df)
    union_df = reduce(DataFrame.unionAll, result)
    union_df=union_df.groupBy('class','classname').avg('speed').sort('class',ascending=True)
    return union_df


In [5]:
w1="hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/processedVehicleData| awk '{print $8}' | awk 'NR!=1' "
w2="hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/lockdownProcessedVehicleData| awk '{print $8}' | awk 'NR!=1' "
L=[w1,w2]
for i in range(0,2):
        j=str(i)
        globals()[f"week{j}"] =  get_Dataframe(L[i])
#average daily speed for both weeks calculated
week0.show()#Normal week
week1.show()#Lockdown week


+-----+---------+-----------------+
|class|classname|       avg(speed)|
+-----+---------+-----------------+
|    1|    MBIKE|85.87613160754174|
|    2|      CAR|92.29858413955627|
|    3|      LGV|94.26504522677455|
|    4|      BUS| 82.5409191870242|
|    5|  HGV_RIG|83.52102209826153|
|    6|  HGV_ART| 83.3820659626253|
|    7|  CARAVAN|83.13369684791348|
+-----+---------+-----------------+

+-----+---------+-----------------+
|class|classname|       avg(speed)|
+-----+---------+-----------------+
|    1|    MBIKE|90.48422177424541|
|    2|      CAR|96.77241797580066|
|    3|      LGV|95.94105077092436|
|    4|      BUS|77.88388951757305|
|    5|  HGV_RIG|82.46460160691093|
|    6|  HGV_ART| 85.2350506215375|
|    7|  CARAVAN| 84.2174328794653|
+-----+---------+-----------------+

